# LOCALTRIAGE - Weekly Insights Report

This notebook generates weekly product issue insights from support tickets.
It performs:
1. Trend analysis of ticket categories and priorities
2. Emerging issue detection using topic clustering
3. Resolution time analysis
4. Actionable recommendations generation

Run this notebook weekly (or schedule via cron/Airflow) to produce insights.

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

# Configuration
os.environ['DB_HOST'] = 'localhost'
os.environ['DB_PORT'] = '5432'
os.environ['DB_NAME'] = 'localtriage'
os.environ['DB_USER'] = 'postgres'
os.environ['DB_PASSWORD'] = 'postgres'

# Report parameters
REPORT_WEEKS = 1  # Number of weeks to analyze
REPORT_DATE = datetime.now()
START_DATE = REPORT_DATE - timedelta(weeks=REPORT_WEEKS)

print(f"Report Period: {START_DATE.strftime('%Y-%m-%d')} to {REPORT_DATE.strftime('%Y-%m-%d')}")

In [ ]:
from ingestion.ingest import DatabaseConnection

# Fetch all tickets in the reporting period
with DatabaseConnection() as conn:
    query = """
        SELECT 
            id,
            subject,
            body,
            category,
            priority,
            status,
            customer_email,
            created_at,
            resolved_at
        FROM tickets
        WHERE created_at >= %s AND created_at <= %s
        ORDER BY created_at;
    """
    df = pd.read_sql(query, conn, params=(START_DATE, REPORT_DATE))

print(f"Loaded {len(df)} tickets for analysis")
df.head()

## 1. Volume & Trend Analysis

In [ ]:
# Daily volume trend
df['date'] = pd.to_datetime(df['created_at']).dt.date
daily_volume = df.groupby('date').size().reset_index(name='count')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Volume over time
axes[0].plot(daily_volume['date'], daily_volume['count'], marker='o', linewidth=2)
axes[0].set_title('Daily Ticket Volume', fontsize=14)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Tickets')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Week-over-week comparison (if enough data)
if len(daily_volume) >= 7:
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.day_name()
    dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dow_counts = df['day_of_week'].value_counts().reindex(dow_order)
    axes[1].bar(dow_counts.index, dow_counts.values, color='steelblue')
    axes[1].set_title('Tickets by Day of Week', fontsize=14)
    axes[1].tick_params(axis='x', rotation=45)
else:
    axes[1].text(0.5, 0.5, 'Insufficient data for\nday-of-week analysis', 
                 ha='center', va='center', fontsize=12)
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

# Summary stats
print(f"\n[CHART] Volume Summary:")
print(f"  Total tickets: {len(df)}")
print(f"  Daily average: {len(df) / max(len(daily_volume), 1):.1f}")
print(f"  Peak day: {daily_volume.loc[daily_volume['count'].idxmax(), 'date']} ({daily_volume['count'].max()} tickets)")

In [ ]:
# Category distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Category pie chart
category_counts = df['category'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))
axes[0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
axes[0].set_title('Ticket Distribution by Category', fontsize=14)

# Priority distribution
priority_order = ['P1', 'P2', 'P3', 'P4']
priority_colors = {'P1': '#dc3545', 'P2': '#fd7e14', 'P3': '#ffc107', 'P4': '#28a745'}
priority_counts = df['priority'].value_counts().reindex(priority_order).fillna(0)
bars = axes[1].bar(priority_counts.index, priority_counts.values, 
                   color=[priority_colors.get(p, 'gray') for p in priority_counts.index])
axes[1].set_title('Ticket Distribution by Priority', fontsize=14)
axes[1].set_ylabel('Count')

# Add value labels
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n[TREND] Category Breakdown:")
for cat, count in category_counts.items():
    pct = count / len(df) * 100
    print(f"  {cat}: {count} ({pct:.1f}%)")

## 2. Emerging Issue Detection

Use topic clustering to identify emerging patterns in tickets.

In [ ]:
# Combine subject and body for clustering
df['text'] = df['subject'].fillna('') + ' ' + df['body'].fillna('')

# TF-IDF vectorization
vectorizer = TfidfVectorizer(
    max_features=500,
    stop_words='english',
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2)
)

if len(df) >= 5:  # Need minimum samples
    tfidf_matrix = vectorizer.fit_transform(df['text'])
    print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
else:
    print("Insufficient data for clustering analysis")
    tfidf_matrix = None

In [ ]:
def get_top_terms(cluster_centers, feature_names, n_terms=10):
    """Extract top terms for each cluster"""
    top_terms = {}
    for i, center in enumerate(cluster_centers):
        top_indices = center.argsort()[-n_terms:][::-1]
        terms = [feature_names[idx] for idx in top_indices]
        top_terms[i] = terms
    return top_terms

if tfidf_matrix is not None and tfidf_matrix.shape[0] >= 5:
    # Determine optimal cluster count (simplified)
    n_clusters = min(5, len(df) // 3)
    n_clusters = max(2, n_clusters)
    
    # K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['cluster'] = kmeans.fit_predict(tfidf_matrix)
    
    # Get top terms per cluster
    feature_names = vectorizer.get_feature_names_out()
    cluster_terms = get_top_terms(kmeans.cluster_centers_, feature_names)
    
    print(f"\n[SEARCH] Identified {n_clusters} Topic Clusters:\n")
    for cluster_id, terms in cluster_terms.items():
        count = (df['cluster'] == cluster_id).sum()
        print(f"Cluster {cluster_id + 1} ({count} tickets):")
        print(f"  Keywords: {', '.join(terms[:5])}")
        
        # Sample ticket from cluster
        sample = df[df['cluster'] == cluster_id].iloc[0]
        print(f"  Example: \"{sample['subject'][:60]}...\"")
        print()

In [ ]:
# Visualize clusters with PCA
if tfidf_matrix is not None and tfidf_matrix.shape[0] >= 5:
    # Reduce to 2D
    pca = PCA(n_components=2, random_state=42)
    coords = pca.fit_transform(tfidf_matrix.toarray())
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(coords[:, 0], coords[:, 1], 
                         c=df['cluster'], cmap='Set2', 
                         alpha=0.7, s=100)
    plt.colorbar(scatter, label='Cluster')
    plt.title('Ticket Topic Clusters (PCA Visualization)', fontsize=14)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.grid(True, alpha=0.3)
    plt.show()

## 3. Resolution Time Analysis

In [ ]:
# Calculate resolution times
resolved = df[df['resolved_at'].notna()].copy()

if len(resolved) > 0:
    resolved['resolution_hours'] = (
        pd.to_datetime(resolved['resolved_at']) - 
        pd.to_datetime(resolved['created_at'])
    ).dt.total_seconds() / 3600
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Resolution time distribution
    axes[0].hist(resolved['resolution_hours'], bins=20, edgecolor='black', alpha=0.7)
    axes[0].axvline(resolved['resolution_hours'].median(), color='red', 
                    linestyle='--', label=f'Median: {resolved["resolution_hours"].median():.1f}h')
    axes[0].set_title('Resolution Time Distribution', fontsize=14)
    axes[0].set_xlabel('Hours')
    axes[0].set_ylabel('Count')
    axes[0].legend()
    
    # Resolution time by priority
    priority_resolution = resolved.groupby('priority')['resolution_hours'].median().reindex(priority_order)
    bars = axes[1].bar(priority_resolution.index, priority_resolution.values,
                       color=[priority_colors.get(p, 'gray') for p in priority_resolution.index])
    axes[1].set_title('Median Resolution Time by Priority', fontsize=14)
    axes[1].set_ylabel('Hours')
    
    # SLA reference lines
    sla_targets = {'P1': 4, 'P2': 24, 'P3': 72, 'P4': 168}
    for i, (p, target) in enumerate(sla_targets.items()):
        if p in priority_resolution.index:
            axes[1].axhline(target, color=priority_colors[p], linestyle=':', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nResolution Time Summary:")
    print(f"  Total resolved: {len(resolved)} ({len(resolved)/len(df)*100:.1f}%)")
    print(f"  Median time: {resolved['resolution_hours'].median():.1f} hours")
    print(f"  Mean time: {resolved['resolution_hours'].mean():.1f} hours")
    print(f"  90th percentile: {resolved['resolution_hours'].quantile(0.9):.1f} hours")
else:
    print("No resolved tickets in the reporting period.")

## 4. Key Insights & Recommendations

In [ ]:
def generate_insights(df):
    """Generate actionable insights from ticket data"""
    insights = []
    
    # High priority ticket analysis
    p1_count = (df['priority'] == 'P1').sum()
    p1_pct = p1_count / len(df) * 100 if len(df) > 0 else 0
    if p1_pct > 10:
        insights.append({
            'type': 'warning',
            'title': 'High P1 Ticket Volume',
            'detail': f'{p1_count} P1 tickets ({p1_pct:.1f}%) - investigate root cause',
            'action': 'Review P1 tickets for common patterns; consider escalation procedures'
        })
    
    # Category concentration
    top_category = df['category'].value_counts().idxmax() if len(df) > 0 else None
    top_cat_pct = df['category'].value_counts().max() / len(df) * 100 if len(df) > 0 else 0
    if top_cat_pct > 40:
        insights.append({
            'type': 'trend',
            'title': f'{top_category} Category Dominance',
            'detail': f'{top_cat_pct:.1f}% of tickets are {top_category}-related',
            'action': f'Investigate {top_category.lower()} issues; consider dedicated support or product fixes'
        })
    
    # Cluster-based insights
    if 'cluster' in df.columns:
        largest_cluster = df['cluster'].value_counts().idxmax()
        cluster_size = df['cluster'].value_counts().max()
        if cluster_size / len(df) > 0.3:
            # Get sample keywords for this cluster
            cluster_samples = df[df['cluster'] == largest_cluster]['subject'].head(3).tolist()
            insights.append({
                'type': 'emerging',
                'title': 'Emerging Issue Pattern Detected',
                'detail': f'Cluster of {cluster_size} similar tickets identified',
                'action': f'Review tickets like: "{cluster_samples[0][:50]}..."'
            })
    
    return insights

insights = generate_insights(df)

print("\n" + "=" * 60)
print("[REPORT] WEEKLY INSIGHTS REPORT")
print("=" * 60)
print(f"Period: {START_DATE.strftime('%Y-%m-%d')} to {REPORT_DATE.strftime('%Y-%m-%d')}")
print(f"Total Tickets Analyzed: {len(df)}")
print("=" * 60)

if insights:
    print("\n[TARGET] KEY INSIGHTS:\n")
    for i, insight in enumerate(insights, 1):
        icon = '[WARN]' if insight['type'] == 'warning' else '[TREND]' if insight['type'] == 'trend' else '[NEW]'
        print(f"{icon} {i}. {insight['title']}")
        print(f"   {insight['detail']}")
        print(f"   → Action: {insight['action']}")
        print()
else:
    print("\n[DONE] No significant issues detected this week.")

print("=" * 60)

In [ ]:
# Export summary to file
report_dir = project_root / 'reports'
report_dir.mkdir(exist_ok=True)

report_filename = f"weekly_insights_{REPORT_DATE.strftime('%Y%m%d')}.md"
report_path = report_dir / report_filename

report_content = f"""# Weekly Support Insights Report

**Period:** {START_DATE.strftime('%Y-%m-%d')} to {REPORT_DATE.strftime('%Y-%m-%d')}  
**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Executive Summary

- **Total Tickets:** {len(df)}
- **Daily Average:** {len(df) / max((REPORT_DATE - START_DATE).days, 1):.1f}
- **Top Category:** {df['category'].value_counts().idxmax() if len(df) > 0 else 'N/A'}
- **P1 Tickets:** {(df['priority'] == 'P1').sum()}

## Category Distribution

| Category | Count | Percentage |
|----------|-------|------------|
"""

for cat, count in df['category'].value_counts().items():
    pct = count / len(df) * 100 if len(df) > 0 else 0
    report_content += f"| {cat} | {count} | {pct:.1f}% |\n"

report_content += "\n## Key Insights\n\n"

for i, insight in enumerate(insights, 1):
    report_content += f"### {i}. {insight['title']}\n\n"
    report_content += f"{insight['detail']}\n\n"
    report_content += f"**Recommended Action:** {insight['action']}\n\n"

if not insights:
    report_content += "No significant issues detected this week.\n\n"

report_content += "---\n*Report generated by LOCALTRIAGE Insights Engine*\n"

report_path.write_text(report_content)
print(f"\n[FILE] Report saved to: {report_path}")

## 5. Save Metrics to Database

Persist weekly metrics for historical tracking.

In [ ]:
# Save aggregated metrics to database
if len(df) > 0:
    metrics = {
        'report_date': REPORT_DATE.date(),
        'period_start': START_DATE.date(),
        'period_end': REPORT_DATE.date(),
        'total_tickets': int(len(df)),
        'p1_count': int((df['priority'] == 'P1').sum()),
        'p2_count': int((df['priority'] == 'P2').sum()),
        'p3_count': int((df['priority'] == 'P3').sum()),
        'p4_count': int((df['priority'] == 'P4').sum()),
        'top_category': df['category'].value_counts().idxmax(),
        'insight_count': len(insights)
    }
    
    print("Weekly Metrics:")
    for key, value in metrics.items():
        print(f"  {key}: {value}")
    
    # Save to database (if table exists)
    try:
        with DatabaseConnection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                INSERT INTO daily_metrics (
                    metric_date, total_tickets, p1_tickets, p2_tickets, p3_tickets, p4_tickets
                ) VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (metric_date) DO UPDATE SET
                    total_tickets = EXCLUDED.total_tickets,
                    p1_tickets = EXCLUDED.p1_tickets,
                    p2_tickets = EXCLUDED.p2_tickets,
                    p3_tickets = EXCLUDED.p3_tickets,
                    p4_tickets = EXCLUDED.p4_tickets;
            """, (
                metrics['report_date'],
                metrics['total_tickets'],
                metrics['p1_count'],
                metrics['p2_count'],
                metrics['p3_count'],
                metrics['p4_count']
            ))
            conn.commit()
            print("\n[DONE] Metrics saved to database.")
    except Exception as e:
        print(f"\n[WARN] Could not save to database: {e}")
else:
    print("No data to save.")

---

## Summary

This notebook has:
1. [DONE] Analyzed ticket volume and trends
2. [DONE] Generated category and priority distributions
3. [DONE] Detected emerging issue patterns via clustering
4. [DONE] Calculated resolution time metrics
5. [DONE] Produced actionable insights
6. [DONE] Exported report to Markdown file
7. [DONE] Persisted metrics to database

**Next Steps:**
- Review insights with product team
- Create tickets for identified issues
- Update KB articles if gaps detected
- Schedule this notebook for weekly execution